In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
batchsize=8

In [3]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'E:\Y R S KUMAR\DESK TOP DATA\things i made\drowsiness\prepared_data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'E:\Y R S KUMAR\DESK TOP DATA\things i made\drowsiness\prepared_data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 61519 images belonging to 2 classes.
Found 15379 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'E:\Y R S KUMAR\DESK TOP DATA\things i made\drowsiness\prepared_data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 8000 images belonging to 2 classes.


In [5]:
bmodel = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 40, 40, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [8]:
checkpoint = ModelCheckpoint(r'E:\Y R S KUMAR\DESK TOP DATA\things i made\drowsiness\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [9]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

C:\Users\PRIYA\AppData\Local\Temp/ipykernel_2456/3918530525.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
7689/7689 [==============================] - ETA: 0s - loss: 0.6946 - accuracy: 0.5035
Epoch 1: val_loss improved from inf to 0.69321, saving model to E:\Y R S KUMAR\DESK TOP DATA\things i made\drowsiness\models\model.h5
7689/7689 [==============================] - 2044s 265ms/step - loss: 0.6946 - accuracy: 0.5035 - val_loss: 0.6932 - val_accuracy: 0.4934 - lr: 0.0010
Epoch 2/5
7689/7689 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5038
Epoch 2: val_loss improved from 0.69321 to 0.69309, saving model to E:\Y R S KUMAR\DESK TOP DATA\things i made\drowsiness\models\model.h5
7689/7689 [==============================] - 2013s 262ms/step - loss: 0.6932 - accuracy: 0.5038 - val_loss: 0.6931 - val_accuracy: 0.5065 - lr: 0.0010
Epoch 3/5
7689/7689 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5061
Epoch 3: val_loss did not improve from 0.69309
7689/7689 [==============================] - 1941s 252ms/step - loss: 0.6931 - accuracy:

In [10]:
loss_tr, acc_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\PRIYA\AppData\Local\Temp/ipykernel_2456/515593128.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_tr, acc_tr = model.evaluate_generator(train_data)


0.5065426826477051
0.6930850148200989


In [11]:
loss_vr, acc_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\PRIYA\AppData\Local\Temp/ipykernel_2456/2909778475.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_vr, acc_vr = model.evaluate_generator(validation_data)


0.5065348744392395
0.693095326423645


In [12]:
loss_test, acc_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

C:\Users\PRIYA\AppData\Local\Temp/ipykernel_2456/2428393938.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_test, acc_test = model.evaluate_generator(test_data)


0.5065426826477051
0.6930850148200989
